# Single objective optimization with (sequential) linear programming and `PESTPP-OPT`

In [ ]:
import os
import sys
sys.path.insert(0,"..")
import numpy as np
import matplotlib.pyplot as plt
import pyemu
print(pyemu.__file__)
import flopy
import platform
from pathlib import Path
import shutil
import pandas as pd
from IPython import display


In [ ]:
display.Image("./mv_schematic.png",width=650) 

## Problem setup
 - two city wells in the south of the domain, in combination need to provide 250,000 ft^3/d of water for a city but would like as much as possible
 - the northern well needs to produce 67,000 ft^3/d although it would be acceptable to produce as little as 50,000 ft^3/d - this is for a fancy brewery making nettle-mead syrup and moss beer #soHipster
 - The two stream gages can experience some depletion, but only up to 30%

In [ ]:
pstroot = 'mv_opt'

In [ ]:
thisdir = os.getcwd()

# Let's start with our iES setup - we need to make some changes to it

In [ ]:
template_ws = Path('./template/')
new_ws = Path('./simple_opt')

In [ ]:
if os.path.exists(new_ws):
    shutil.rmtree(new_ws)
shutil.copytree(template_ws, new_ws)

In [ ]:
pst = pyemu.Pst(str(new_ws / 'at.pst'))

### let's start with the base realization parameters for now

In [ ]:
pst.parrep('./master_ies_simple/at.3.base.par')

# let's check out the parameter setup and look for WEL-related ones

In [ ]:
pars = pst.parameter_data

In [ ]:
pars

In [ ]:
[i for i in pars.parnme if 'wflux' in i]

### what about the template files?

In [ ]:
pst.template_files

### we have some bespoke TPL files lurking in there though

In [ ]:
[print(i) for i in new_ws.glob('*.tpl')];

### and - we have a WEL one. Let's have a look and, if promising, add it

In [ ]:
[print(i.strip()) for i in open('simple_opt/at.wel.tpl').readlines()];

In [ ]:
[print(i.strip()) for i in open('simple_opt/at.wel').readlines()];

### We can just add parameters to the PST object by letting pyemu read the TPL file

In [ ]:
pst.add_parameters('simple_opt/at.wel.tpl', pst_path='.')

In [ ]:
pars = pst.parameter_data

In [ ]:
pars.loc[pars.index.str.contains('wflux')]

In [ ]:
pst.template_files

# Now we can redefine these wells as _decision variables_ recalling the problem statement:
 - the wells at the higher `i` (row) value (32 and 34) in combination need to provide 268,000 ft^3/d of water for a city
 - the well at `i=5` needs to produce 67,000 ft^3/d although it would be acceptable to produce as little as 50,000 ft^3/d - this is for a fancy brewery making nettle-mead syrup and moss beer #soHipster

### first we need to fix all the parameters that will not be decision variables

In [ ]:
pars['partrans'] = 'fixed'

### then let's assign the decision variables to a group and flip the sign

In [ ]:
pars.loc[pars.index.str.contains('wflux_k:4_'), 'pargp'] = 'well_decision_variables'
pars.loc[pars.index.str.contains('wflux_k:4_'), 'scale'] = -1 # let's flip the sign on pumping so more intuitive
pars.loc[pars.index.str.contains('wflux_k:4_'), 'partrans'] = 'none' # we want zero to be acceptable value, so no log xform
pars.loc[pars.index.str.contains('wflux_k')]

### let's set bounds on the pumping

In [ ]:
pars.loc[pars.index.str.contains('wflux_k:4_i:3'), 'parlbnd'] = 0
pars.loc[pars.index.str.contains('wflux_k:4_i:3'), 'parubnd'] = 268000
pars.loc[pars.index.str.contains('wflux_k:4_i:5'), 'parlbnd'] = 50000
pars.loc[pars.index.str.contains('wflux_k:4_i:5'), 'parubnd'] = 67000

pars.loc[pars.index.str.contains('wflux_k:4_i:5'), 'parval1'] = 60000
pars.loc[pars.index.str.contains('wflux_k:4_i:3'), 'parval1'] = 268000/2



### now constrain the wells in rows 32 and 34 to sum to 250,000 or greater

In [ ]:
pst.add_pi_equation(par_names=['wflux_k:4_i:32_j:5', 'wflux_k:4_i:34_j:15'],
                rhs=250000,
                weight=1,
                pilbl='city_wells',
                obs_group='greater_than_pumping')



### We have contstraints also on the observations - particularly streamflow and lake flux

In [ ]:
obs = pst.observation_data
obs

In [ ]:
obs.weight=0

In [ ]:
obs.loc[obs.obgnme=='rivgroup', 'obgnme'] = 'less_than_riv'
obs.loc[obs.obgnme=='less_than_riv', 'weight'] = 1.0
obs.loc[obs.obgnme=='less_than_riv', 'obsval'] *= .7


obs

## now an objective function - let's maximize total pumping

In [ ]:
wellnames = [i for i in pars.index if 'wflux' in i]
wellnames

In [ ]:
pst.add_pi_equation(wellnames, # parameter names to include in the equation
                    pilbl="obj_well",  # the prior information equation name
                    obs_group="greater_than") # note the "greater_" prefix.

In [ ]:
pst.prior_information

In [ ]:
pst.pestpp_options["opt_direction"] = "max"
pst.pestpp_options["opt_objective_function"] = "obj_well"

In [ ]:
pst.rectify_pgroups()
pst.parameter_groups

In [ ]:
pst.parameter_groups.loc["well_decision_variables","inctyp"] = "absolute"
pst.parameter_groups.loc["well_decision_variables","derinc"] = 5000 #remember these are multipliers!
pst.parameter_groups.loc["well_decision_variables","derinclb"] = 500

In [ ]:
pst.pestpp_options["opt_dec_var_groups"] = "well_decision_variables"

In [ ]:
pst.control_data.noptmax = 0
pst.write(str(new_ws / f'{pstroot}.pst'), version=2)

In [ ]:
os.chdir(new_ws)
pyemu.os_utils.run(f'pestpp-opt {pstroot}.pst')
os.chdir(thisdir)

In [ ]:
pst.control_data.noptmax = 3
pst.write(str(new_ws / f'{pstroot}.pst'), version=2)

In [ ]:
os.chdir(new_ws)
pyemu.os_utils.run(f'pestpp-opt {pstroot}.pst')
os.chdir(thisdir)

# let's look at the results

In [ ]:
obgroups = ['greater_than','less_than_riv','greater_than_pumping']
estimated = [pd.read_csv(e, sep=r'\s+', skiprows=3, index_col=0)
            for e in new_ws.glob('*est.rei')]
simulated = [pd.read_csv(e, sep=r'\s+', skiprows=3, index_col=0)
            for e in new_ws.glob('*sim.rei')]
pars = [pd.read_csv(p, sep=r'\s+', skiprows=1, 
                    names=['parname','parval','scale','offset'], index_col=0)
            for p in new_ws.glob('*opt.?.par')]

estimated = [e.loc[e.Group.isin(obgroups)] for e in estimated]
simulated = [s.loc[s.Group.isin(obgroups)] for s in simulated]
par0 = pd.read_csv(new_ws / 'mv_opt.par_data.csv', index_col=0)
pars = [par0.loc[par0.index.str.startswith('wflux'),'parval1'].to_frame().rename(
        columns={'parval1':'parval'})] + \
        [p.loc[p.index.str.startswith('wflux'),'parval'].to_frame()
           for p in pars]

## how do the decision variables look?

In [ ]:
for cp in pars[0].index:
    par_evol = pd.DataFrame(index=range(len(pars)),
                data = {cp:[p.loc[cp, 'parval'] 
                            for p in pars]}
    )
    par_evol.plot.bar(figsize=(4,4))
    

In [ ]:
### how about objective function?

In [ ]:
obfun = pd.DataFrame(index=range(len(estimated)),
            data={
                'estimated_obfun':
                    [e.loc[e.Group=='greater_than', 'Modelled'].values[0]
                    for e in estimated],
                'simulated_obfun': 
                    [s.loc[s.Group=='greater_than','Modelled'].values[0]
                    for s in simulated]
            })
obfun.plot.bar(figsize=(4,4))

### and constraints?

In [ ]:
for cob in estimated[0].loc[estimated[0].Group=='less_than_riv'].index:
    rivob = pd.DataFrame(index=range(len(estimated)),
                data={
                    f'estimated_{cob}':
                        [e.loc[cob, 'Modelled']
                        for e in estimated],
                    f'simulated_{cob}': 
                        [s.loc[cob,'Modelled']
                        for s in simulated]
                })
    rivob.plot.bar(figsize=(4,4))

### but no, really, they are a lil different!

In [ ]:
rivob